### Exercise 1: Threaded Prime Number Checker

Write a Python program that checks whether a given range of numbers contains prime numbers. Divide the range among multiple threads to parallelize the prime checking process. Each thread should be responsible for checking a subset of the range, and the main program should print the list of prime numbers found.

In [ ]:
import threading

def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

def check_primes(start, end, result_list):
    for number in range(start, end):
        if is_prime(number):
            result_list.append(number)

def threaded_prime_checker(start, end, num_threads=4):
    threads = []
    result = []
    step = (end - start) // num_threads
    partial_results = [[] for _ in range(num_threads)]

    for i in range(num_threads):
        sub_start = start + i * step
        sub_end = end if i == num_threads - 1 else sub_start + step
        thread = threading.Thread(target=check_primes, args=(sub_start, sub_end, partial_results[i]))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    for part in partial_results:
        result.extend(part)

    print("Prime numbers:", sorted(result))

# Example usage
threaded_prime_checker(1, 100, num_threads=4)


### Exercise 2: Threaded File Processing

Write a program that reads a large text file containing lines of text. Implement a threaded solution to count the occurrence of each word in the file. Each thread should process a portion of the file, and the main program should display a summary of word occurrences across all threads.

In [ ]:
import threading
from collections import defaultdict

def count_words(lines, result_dict, lock):
    local_count = defaultdict(int)
    for line in lines:
        words = line.strip().split()
        for word in words:
            local_count[word.lower()] += 1

    # Safely merge local count into shared dictionary
    with lock:
        for word, count in local_count.items():
            result_dict[word] += count

def threaded_word_counter(filename, num_threads=4):
    with open(filename, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    total_lines = len(lines)
    step = total_lines // num_threads
    threads = []
    word_counts = defaultdict(int)
    lock = threading.Lock()

    for i in range(num_threads):
        start = i * step
        end = total_lines if i == num_threads - 1 else start + step
        thread = threading.Thread(target=count_words, args=(lines[start:end], word_counts, lock))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    print("Word Count Summary:")
    for word, count in sorted(word_counts.items()):
        print(f"{word}: {count}")

# Example usage
# Save a test file named 'sample.txt' before running this
# threaded_word_counter('sample.txt', num_threads=4)
